In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
mnist_datasets, mnist_info = tfds.load(name='mnist',with_info=True, as_supervised=True )

In [3]:
mnist_train, mnist_test = mnist_datasets['train'], mnist_datasets['test']

num_validation_samples = 0.1* mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image,tf.float32)
    image /=255.
    return image, label


scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


## Shuffling and batching

In [4]:
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)


validation_inputs , validation_targets = next(iter(validation_data))



### Outline The MODEL

In [8]:
input_size = 784
output_size = 10
hidden_layer_size= 200


model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape = (28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

### Choose the optimizer and loss function

In [9]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training

In [10]:
num_epochs = 5


model.fit(train_data, epochs= num_epochs, validation_data= (validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 11s - loss: 0.2757 - accuracy: 0.9207 - val_loss: 0.1403 - val_accuracy: 0.9600
Epoch 2/5
540/540 - 10s - loss: 0.1077 - accuracy: 0.9679 - val_loss: 0.0933 - val_accuracy: 0.9747
Epoch 3/5
540/540 - 11s - loss: 0.0709 - accuracy: 0.9785 - val_loss: 0.0765 - val_accuracy: 0.9777
Epoch 4/5
540/540 - 10s - loss: 0.0548 - accuracy: 0.9827 - val_loss: 0.0571 - val_accuracy: 0.9827
Epoch 5/5
540/540 - 12s - loss: 0.0410 - accuracy: 0.9873 - val_loss: 0.0422 - val_accuracy: 0.9865


###  Test the Model

In [12]:
test_loss, test_accuracy = model.evaluate(test_data)

print('Test loss: {0: .2f}.  Test accuracy: {1: .2f}%' .format(test_loss, test_accuracy*100.))

1/1 [==============================] - 2s 2s/step - loss: 0.0709 - accuracy: 0.9793
Test loss:  0.07.  Test accuracy:  97.93%
